<a href="https://colab.research.google.com/github/karoldem/travel-season/blob/main/travel_season.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
europe = "https://raw.githubusercontent.com/karoldem/travel-season/main/List_of_cities_by_average_temperature_1.csv"
europe = pd.read_csv(europe)

listOfMonths = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [14]:
import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * numpy.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

In [26]:
import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "offset" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 0., guess_offset])

    def sinfunc(t, A, p, c):  return A * numpy.sin(0.5235987755982988*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, p, c = popt
    f = 0.5235987755982988/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(0.5235987755982988*t + p) + c
    return {"amp": A,  "phase": p, "offset": c, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

In [33]:
data = {}

for i in europe.index:
    k = []

    for j in listOfMonths:
        k.append ( float(europe.loc[i][j].split('\n')[0]) )

    temporaryNvmVaiable = fit_sin([0,1,2,3,4,5,6,7,8,9,10,11], k)
    toAddtemporaryVariable = {}
    toAddtemporaryVariable['temperature'] = {}
    toAddtemporaryVariable['temperature']['amplitude'] = temporaryNvmVaiable['amp']
    toAddtemporaryVariable['temperature']['average'] = temporaryNvmVaiable['offset']
    toAddtemporaryVariable['temperature']['phase'] = temporaryNvmVaiable['phase']

    data[(europe.loc[i]['City'], europe.loc[i]['Country'])] = toAddtemporaryVariable
    
    break

data

{('Algiers',
  'Algeria'): {'temperature': {'amplitude': 6.936254984230482,
   'average': 17.441666666666052,
   'phase': -1.9034829889098737}}}

0.5235987755982988